In [11]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')
X_predict=pd.read_csv('../data/processed/X_predict.csv')
df_predict=pd.read_csv('../data/processed/df_predict.csv')

In [12]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.1-internal" 2018-10-16; OpenJDK Runtime Environment (build 11.0.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyitqwrgy
  JVM stdout: /tmp/tmpyitqwrgy/h2o_carpiero_started_from_python.out
  JVM stderr: /tmp/tmpyitqwrgy/h2o_carpiero_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_carpiero_jbnj0y
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.686 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [13]:
X_Y=pd.concat([X,Y], axis=1)


In [14]:
x= X.columns.to_list()
y = "price"
x

['depth',
 'table',
 'x',
 'y',
 'z',
 'cut_int',
 'color_int',
 'clarity_int',
 'raw_density',
 'carat_sqr']

In [ ]:
train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(seed=32,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |██████████████████████████████████████████████

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [7]:
predict_X=h2o.H2OFrame(X_predict)
predictions_competition= aml.leader.predict(predict_X)
predictions_competition=predictions_competition.as_data_frame()


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [8]:

submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition['predict']})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)



In [9]:
h2o.cluster().shutdown()

H2O session _sid_94e3 closed.


In [10]:
submission.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,3923.079634
std,3892.928525,3948.298431
min,0.000000,319.120766
25%,3371.000000,910.196227
50%,6742.000000,2428.529442
75%,10113.000000,5302.686349
max,13484.000000,18355.638023
